In [1]:
#Подключение Torch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.1.2%2Bcu121-cp311-cp311-win_amd64.whl (2474.0 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.2%2Bcu121-cp311-cp311-win_amd64.whl (5.6 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.1.2%2Bcu121-cp311-cp311-win_amd64.whl (4.0 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.0-py3-none-any.whl (2.0 MB)
  Using cached https://download.pytorch.org/whl/sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2023.4.0-py3-none-any.whl (153 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.9.0-py3-none-any.whl (9.7 kB)
  Using cached https://download.pytorch.org/whl/Pillow-9.3.0-cp311-cp311-win_amd64.whl (2.5 MB)
  Using cached https://download.pytorch.org/whl/numpy-1

You should consider upgrading via the 'C:\Users\gibad\PycharmProjects\Yolo8nGD\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import torch
print(torch.cuda.is_available())    #Проверка доступности для обучения видеокарты

True


In [3]:
#Установка пакетов YOLO и ClearML
!pip install ultralytics
!pip install clearml
#Библиотека для очистки экрана
from IPython import display
# Очистить экран.
display.clear_output()
print('Пакеты YOLO и ClearML установлены')

Пакеты YOLO и ClearML установлены


In [2]:
#Импорт библиотек
from ultralytics import YOLO
from clearml import Task

In [3]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=DQOUT5JZW5R9B632MD9C
%env CLEARML_API_SECRET_KEY=E4uim6ziCuKyOj7l4qmTBu1mGoC6MudDrMGstFF5wTZoaYEq7m


env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=DQOUT5JZW5R9B632MD9C
env: CLEARML_API_SECRET_KEY=E4uim6ziCuKyOj7l4qmTBu1mGoC6MudDrMGstFF5wTZoaYEq7m


In [4]:
#Создание задачи в ClearML
task = Task.init(
    project_name='Gun_detection',
    task_name='Train_YOLOv8n_200',
    tags=['Train','YOLOv8n'])

ClearML Task: created new task id=47255dcf022b480b8d0e736ed1eae154
2024-01-18 22:40:44,342 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/2262de1bd95b4c66b95f82fb49d5b21f/experiments/47255dcf022b480b8d0e736ed1eae154/output/log


In [5]:
dataset_path = 'E:\dataset.zip'
data_path = 'E:\dataset'

In [6]:
#Выгрузка датасета в задачу ClearML
task.upload_artifact(name='dataset', artifact_object=dataset_path)

True

In [7]:
# Выбор типа модели YOLO.
model = 'yolov8n' # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
# Создание параметра типа модели в эксперименте ClearML.
task.set_parameter('model', model)

In [8]:
# Создание словаря переменных используемых в YOLO.
# Через словарь есть возможность передавать данные из ClearML при клонировании эксперимента с новыми параметрами.
args = dict(data = data_path + '/data.yaml',                          #путь к файлу данных, например coco128.yaml
            epochs = 200,	                                          #количество эпох для тренировок
            patience =	100,	                                      #эпохи, чтобы дождаться отсутствия заметного улучшения для раннего прекращения тренировок
            batch = 16,                                               #количество изображений в пакете (-1 для AutoBatch)
            imgsz	= 640,	                                          #размер входных изображений как целое число
            optimizer =	'Adam',	                                      #используемый оптимизатор, choice=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
            lr0	= 1E-3,                                               #начальная скорость обучения (т.е. SGD=1E-2, Adam=1E-3)
            lrf	= 1E-3,	                                              #конечная скорость обучения (lr0 * lrf)
            verbose = True,	                                          #печатать ли подробный вывод
            val =	True,	                                          #проверить/проверить во время обучения
            plots = True
            )

In [9]:
# Отправка параметров ClearML.
task.connect(args)
# Загрузка модели
train_model = YOLO('yolov8n.yaml')
# Запуск обучения
result = train_model.train(**args)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

train: Scanning E:\dataset\train\labels.cache... 5406 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5406/5406 [00:00<?, ?it/s]
val: Scanning E:\dataset\train\labels.cache... 5406 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5406/5406 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: Adam(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200       2.4G      2.802      2.987      3.026         45        640: 100%|██████████| 338/338 [00:34<00:00,  9.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 169/169 [00:26<00:00,  6.43it/s]


                   all       5406      12905      0.189      0.262      0.102     0.0287

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    200/200      2.39G     0.7433     0.3749      1.169         30        640: 100%|██████████| 338/338 [00:28<00:00, 11.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 169/169 [00:20<00:00,  8.23it/s]

                   all       5406      12905      0.982      0.983      0.993      0.814



200 epochs completed in 2.875 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.3MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.2 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8191MiB)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 169/169 [00:23<00:00,  7.22it/s]


                   all       5406      12905      0.982      0.983      0.993      0.814
                  guns       5406       5840      0.975      0.973      0.992      0.766
                person       5406       7065      0.988      0.993      0.995      0.862
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\train2
2024-01-19 01:35:39,150 - clearml.storage - INFO - Starting upload: runs\detect\train2\weights\best.pt => https://files.clear.ml/Gun_detection/Train_YOLOv8n_200.47255dcf022b480b8d0e736ed1eae154/models/best.pt


In [10]:
# Load a pretrained YOLOv8n model
model = 'C:\\Users\gibad\PycharmProjects\Yolo8nGD\\runs\detect\\train2\weights\\best.pt'

# Define path to video file
source = 'D:\Predict\\test_video.mp4'
task.upload_artifact(name='test_video', artifact_object=source)

True

In [11]:
!yolo task=detect mode=predict model=$model source=$source show=True conf=0.6 imgsz=640

Ultralytics YOLOv8.1.2 рџљЂ Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8191MiB)
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (1/4362) D:\Predict\test_video.mp4: 384x640 1 guns, 1 person, 84.0ms
video 1/1 (2/4362) D:\Predict\test_video.mp4: 384x640 1 person, 5.0ms
video 1/1 (3/4362) D:\Predict\test_video.mp4: 384x640 1 person, 6.0ms
video 1/1 (4/4362) D:\Predict\test_video.mp4: 384x640 1 person, 7.0ms
video 1/1 (5/4362) D:\Predict\test_video.mp4: 384x640 1 person, 6.0ms
video 1/1 (6/4362) D:\Predict\test_video.mp4: 384x640 1 person, 6.0ms
video 1/1 (7/4362) D:\Predict\test_video.mp4: 384x640 1 person, 5.0ms
video 1/1 (8/4362) D:\Predict\test_video.mp4: 384x640 1 person, 6.0ms
video 1/1 (9/4362) D:\Predict\test_video.mp4: 384x640 1 person, 6.0ms
video 1/1 (10/4362) D:\Predict\test_video.mp4: 384x640 1 person, 5.0ms
video 1/1 (11/4362) D:\Predict\test_video.mp4: 384x640 1 person, 5.0ms
video 1/1 (12/4362) D:\Predict\t

In [12]:
task.upload_artifact(name='test_video_predict', artifact_object='C:\\Users\gibad\PycharmProjects\Yolo8nGD\\runs\detect\predict\\test_video.avi')

True

In [13]:
task.close()